# EEG-to-Text HMM Pipeline - Google Colab (GitHub Version)

This notebook runs the EEG-to-Text pipeline using:
- **Code**: From GitHub repository
- **Data**: From your Google Drive

**Expected time**: 30-60 minutes

**Expected accuracy**: 55-65%

---

## ⚠️ Before Starting

1. **Enable GPU**: Runtime → Change runtime type → GPU (T4)
2. **Upload data to Drive**: Create folder `ML_Project_Data/processed_data/` with all CSV files

---

## Step 1: Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
print("✓ Google Drive mounted successfully!")

## Step 2: Clone GitHub Repository

In [ ]:
# Clone the repository
!git clone https://github.com/Tejas-Chakkarwar/ML-Project-Data.git

# Change to project directory
%cd ML-Project-Data

# List files to verify
!ls -la
print("\n✓ Repository cloned successfully!")

## Step 3: Install Dependencies

In [ ]:
!pip install -q torch numpy pandas
print("✓ Dependencies installed!")

## Step 4: Link Your Google Drive Data

**⚠️ IMPORTANT**: Update `DRIVE_DATA_PATH` if your folder name is different!

In [ ]:
import os

# ========================================
# UPDATE THIS PATH TO MATCH YOUR GOOGLE DRIVE FOLDER
# ========================================
DRIVE_DATA_PATH = '/content/drive/MyDrive/ML_Project_Data/processed_data'

# Create symbolic link to your data
if os.path.exists('processed_data'):
    !rm -rf processed_data

!ln -s "{DRIVE_DATA_PATH}" processed_data

# Verify data is accessible
print("\n📁 Checking data files...")
!ls -lh processed_data/ | head -10

print("\n📊 Checking sentence mapping...")
!wc -l processed_data/sentence_mapping.csv

print("\n✓ Data linked successfully!")

## Step 5: Configure GPU

In [ ]:
# Update config to use GPU
!sed -i "s/CNN_DEVICE = 'cpu'/CNN_DEVICE = 'cuda'/" src/config.py

# Verify GPU is available
import torch
print(f"GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️ WARNING: GPU not available! Training will be slow.")
    print("Go to: Runtime → Change runtime type → GPU")

## Step 6: Quick Test (Optional - 2-3 minutes)

Run this to verify everything works before full training.

In [ ]:
!python main.py --quick-test

## Step 7: Full Training (30-60 minutes) ⭐

**This is the main training step!**

Expected accuracy: **55-65%**

In [ ]:
# Recommended settings for best results
!python main.py \
  --cnn-epochs 5 \
  --hmm-states 5 \
  --num-aug 2 \
  --save-models \
  --verbose

## Step 8: Save Models to Google Drive

This ensures your models persist after the Colab session ends.

In [ ]:
# Create checkpoints folder in Google Drive
!mkdir -p /content/drive/MyDrive/ML_Project_Data/checkpoints

# Copy trained models
!cp -v checkpoints/*.pth /content/drive/MyDrive/ML_Project_Data/checkpoints/ 2>/dev/null || echo "No .pth files"
!cp -v checkpoints/*.pkl /content/drive/MyDrive/ML_Project_Data/checkpoints/ 2>/dev/null || echo "No .pkl files"

print("\n✓ Models saved to Google Drive!")
print("Location: /MyDrive/ML_Project_Data/checkpoints/")

# List saved models
!ls -lh /content/drive/MyDrive/ML_Project_Data/checkpoints/

## Step 9: Download Models (Optional)

Download models to your local machine.

In [ ]:
from google.colab import files

# Download CNN encoder
if os.path.exists('checkpoints/cnn_encoder.pth'):
    files.download('checkpoints/cnn_encoder.pth')
    print("✓ CNN encoder downloaded")

# Download HMM models
if os.path.exists('checkpoints/hmm_models.pkl'):
    files.download('checkpoints/hmm_models.pkl')
    print("✓ HMM models downloaded")

---

## 🎯 Alternative Training Options

### Memory-Efficient Version (if OOM errors)

In [ ]:
!python main_memory_efficient.py \
  --num-aug 1 \
  --cnn-batch-size 4 \
  --save-models

### Optimized for Best Accuracy (60-90 minutes)

In [ ]:
!python main.py \
  --cnn-epochs 10 \
  --hmm-states 7 \
  --num-aug 3 \
  --cnn-batch-size 16 \
  --save-models

### Resume from Checkpoint

In [ ]:
# Resume from Google Drive checkpoint
!python main.py \
  --resume /content/drive/MyDrive/ML_Project_Data/checkpoints/cnn_encoder.pth \
  --num-aug 2 \
  --save-models

---

## 🔧 Troubleshooting

### Check GPU Usage

In [ ]:
!nvidia-smi

### Check Data Path

In [ ]:
# List your Google Drive folders
!ls -la /content/drive/MyDrive/

# Check if data exists
!ls -lh processed_data/ | head -5

### Check Available Memory

In [ ]:
!free -h

---

## 📊 Expected Results

| Configuration | Accuracy | Time |
|--------------|----------|------|
| Quick test | 30-40% | 2-3 min |
| Default | 55-65% | 30-45 min |
| Optimized | 60-70% | 60-90 min |

**Random guessing**: 0.29% (1/344)

**Your model**: 55-65% = **172-241x better than random!** 🎉

---

## ✅ Done!

Your models are saved in:
- **Colab session**: `/content/ML-Project-Data/checkpoints/`
- **Google Drive**: `/MyDrive/ML_Project_Data/checkpoints/`

They will persist in Google Drive even after the session ends!